# Berdasarkan textOriginal

In [8]:
from pymongo import MongoClient

tahun = 2023

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Sesuaikan dengan URI MongoDB Anda
db = client['youtube_rewind_indonesia']  # Ganti dengan nama database Anda

# Koleksi
labeling_2015 = db[f'labeling_{tahun}']
preprocessing_new_2015 = db[f'preprocessing_new_{tahun}']
labeling_new_2015 = db[f'labeling_new_{tahun}']

# Mengambil semua data dari labeling_tahun
labeling_data = list(labeling_2015.find())

for label in labeling_data:
    text_original = label['textOriginal']
    
    # Mencari data yang sesuai di preprocessing_new_tahun
    preprocessed_data = preprocessing_new_2015.find_one({'textOriginal': text_original})
    
    if preprocessed_data:
        # Menggabungkan data dan mempertahankan _id dari preprocessing_new_tahun
        merged_data = {**preprocessed_data, **label}
        
        # Memasukkan data yang sudah digabungkan ke labeling_new_tahun
        labeling_new_2015.insert_one(merged_data)

print("Proses penggabungan selesai.")


Proses penggabungan selesai.


# Berdasarkan _id

In [ ]:
from pymongo import MongoClient

tahun = 2015

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['youtube_rewind_indonesia']

# Koleksi
preprocessing_collection = db[f'preprocessing_{tahun}']
cleansing_collection = db[f'cleansing_{tahun}']

# Mengambil semua data dari koleksi preprocessing
preprocessing_data = list(preprocessing_collection.find())

# Inisialisasi list untuk menyimpan pasangan dokumen yang ditemukan
matched_documents = []

for preprocessed_doc in preprocessing_data:
    doc_id = preprocessed_doc['_id']
    
    # Mencari dokumen yang sesuai di koleksi cleansing berdasarkan _id
    cleansing_doc = cleansing_collection.find_one({'_id': doc_id})
    
    if cleansing_doc:
        # Menggabungkan data dan menambahkan ke list matched_documents
        matched_data = {**cleansing_doc, **preprocessed_doc}
        matched_documents.append(matched_data)

# Menampilkan jumlah pasangan yang ditemukan
print(f"Jumlah pasangan dokumen yang ditemukan: {len(matched_documents)}")

# Menyimpan pasangan dokumen yang ditemukan ke koleksi baru
matched_collection = db['full_labeling_all_new']
matched_collection.insert_many(matched_documents)

print("Proses pencocokan dan penyimpanan selesai.")
